# Design of a Tension Member and End Connections

| Property | Value |
|----------|:-------:|
|  $T_f$   | 800 kN|
| Angles   | 300W  |
| Plates   | 300W |
| HSS      | 350W |
| W        | A992 |
| Bolts    | A325 3/4" |
| Holes    | 22 mm punched |
| Electrodes | E49xx |

In [1]:
phi = 0.9
phiu = 0.75
phib = 0.8
phiw = 0.67

aFy = 300   # angles
aFu = 440
pFy = 300   # plates
pFu = 440
hFy = 350   # HSS
hFu = 450
wFy = 345   # W shapes
wFu = 450
bFu = 825   # bolt
Xu = 490    # weld
bd = 25.4 * 0.75   # 3/4" bolt
hd = 22 + 2       # in 22mm punched holes

In [2]:
Tf = 800

In [3]:
from sst import SST
sst = SST()

Tension brace will be bolted into frame.  Net area fracture is often the governing case.  Reasonable
estimates are $A_{n} = 0.9 A_g$ and $A_ne = 0.85 A_n$.  Therefore, a first estimate of cross-sectional
area required is, without differentiating material strengths, is:

$$
\phi_u \times 0.85 \times 0.9 A_g \times Fu \ge T_f
$$

$$
A_g \ge \frac{T_f}{\phi_u 0.85 \times 0.9 Fu}
$$

In [4]:
Tf/(phi*0.85*0.9*450*1E-3)

2582.1027999677235

Therefore, look for a square HSS of at least this cross-sectional area:

>   _Gentle Reader:_ If you find the following 'python-pandas magic' hard to understand,
>   don't worry about it.  Its just the digital equivalent of searching through
>   the properties and dimensions tables in Part 6 of the CISC handbook of Steel Construction.

In [5]:
# load a table of all HSS sections, sorted by increasing mass
hss = sst.section_tables(['HS'],'Mass')

# search for those shapes whose area meets the limit and whose B and D dimensions are equal (square)
# Then extract three columns from the result and display the first 10 shapes.
hss.query('A>2500 & B==D')[['A','T','Mass']].head(10)

,A,T,Mass
Dsg,,,
HS114x114x6.4,2640.0,6.35,20.7
HS152x152x4.8,2760.0,4.78,21.7
HS89x89x9.5,2790.0,9.53,21.9
HS102x102x8.0,2820.0,7.95,22.1
HS127x127x6.4,2960.0,6.35,23.2
HS114x114x8.0,3220.0,7.95,25.3
HS178x178x4.8,3250.0,4.78,25.5
HS102x102x9.5,3280.0,9.53,25.7
HS152x152x6.4,3610.0,6.35,28.3


Probably do not want the thinnest section (because of welding considerations). 
Also, a larger dimension may be easier to connect.
And leave a bit of allowance 
over the minimum.  Therefore:

### Try: HSS 127x127x6.4

In a configuration similar to this:

![Photo](170-DSCN0768.JPG)

In [35]:
##angle=sst.section_tables(['L'],'Mass')
##angle.query("Avl!='*'&A>2500/4")[['A','T','Mass']].head(10)

In [36]:
##angle=sst.section_tables(['W'],'Mass')
##angle.query("Avl!='*'&A>2500")[['Dsg','A','T','Mass']].head(10)

#### Bolting Requirements

Again, because net area considerations usually govern we should start by selecting a bolt configuration (number of bolts and layout).  That will allow us to decide gusset plate thickness, which is important because the insert
plate welded into the HSS must be the same thickness, and that affects the strength of the HSS.

In [42]:
Ab = 3.14159*bd*bd/4   # bolt shear area
vr1 = 0.60*phib*1*2*Ab*bFu * 0.7 * 1E-3   # factored shear strength of 1 bolt, double shear, threads intercepted
Tf/vr1                 # number of bolts required

5.062759424997536

Therefore, use 6 bolts.  But should they be in a 3x2 or a 2x3 arrangement?

In [45]:
hss.loc['HS127x6.4']

Prp           NaN
Dsg     HS127x6.4
Avl             +
Shp            17
Grp           NaN
Use             2
D             127
B             NaN
T            6.35
Tdes         6.35
RI            NaN
RO            NaN
BT            NaN
DT             20
Dnom          127
Mass         18.9
A            2410
Ix       4.39e+06
Sx          69200
Rx           42.7
Zx          92500
Iy       4.39e+06
Sy          69200
Ry           42.7
Zy          92500
J        8.78e+06
Crt          1210
C          138000
SA            0.4
Ds_i    HS5x0.250
Dn_i            5
Wt_i         12.7
Name: HS127x6.4, dtype: object